`Course Instructor`: **John Chiasson**

`Author (TA)`: **Ruthvik Vaila**

# Notes:
* In this notebook we shall load a large `NumPy` array directly into RAM to train a model.
* While the model is training keep an eye on the time taken and RAM usage of your machine.
* Tested on `Python 3.7.5` with `Tensorflow 1.15.0` and `Keras 2.2.4`. 
* Tested on `Python 2.7.17` with `Tensorflow 1.15.3` and `Keras 2.2.4`. 

# Imports

In [1]:
import sys, os
sys.version

'3.7.5 (default, Nov  7 2019, 10:50:52) \n[GCC 8.3.0]'

In [2]:
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #setting it to -1 hides the GPU.
#tf.compat.v1.enable_eager_execution()
from tensorflow.python.client import device_lib
import numpy as np
import IPython
import sys, pickle, os
import h5py, time, inspect
import IPython.display as display
import keras, warnings
warnings.filterwarnings(action='once')
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
# this make sure thaat if using a gpu total gpu memory is not gobbled
# up by tensorflow and allows growth as required

Using TensorFlow backend.


In [3]:
device_lib.list_local_devices()

/home/visionteam/python37_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6048882361621787560,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1315446857136066081
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12231405424167414289
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7458901197
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 993663886900078102
 physical_device_desc: "device: 0, name: GeForce RTX 2080 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
tf.__version__

'1.15.0'

In [5]:
keras.__version__

'2.2.4'

# Load the large training and testing `NumPy` arrays

In [6]:
filename = 'data/emnist_train_x.h5'
with h5py.File(filename, 'r') as hf:
    train_x = hf['pool1_spike_features'][:]

filename = 'data/emnist_test_x.h5'
with h5py.File(filename, 'r') as hf:
    test_x = hf['pool1_spike_features'][:]

print('Train data shape:{}'.format(train_x.shape))
print('Test data shape:{}'.format(test_x.shape))

train_x = list(train_x) #convert 2D numpy array to a list of 1D numpy arrays 
test_x = list(test_x)

Train data shape:(112799, 3630)
Test data shape:(18800, 3630)


# Load the large training and testing label `NumPy` arrays

In [7]:
filename = 'data/emnist_train_y.pkl'
filehandle = open(filename, 'rb')
train_y = pickle.load(filehandle)
filehandle.close()

filename = 'data/emnist_test_y.pkl'
filehandle = open(filename, 'rb')
test_y = pickle.load(filehandle)
filehandle.close()

print('Train labels shape:{}'.format(train_y.shape))
print('Test labels shape:{}'.format(test_y.shape))

train_y = train_y.tolist() #convert 2D numpy array to a list of 1D numpy arrays 
test_y = test_y.tolist()

Train labels shape:(112799,)
Test labels shape:(18800,)


# Train a small NN model using `tf.keras.model.fit` 
* A simple fully connected neural network with the structure
* 3630 -> 1500 -> 47
* `Adam optimizer` and `Cross Entropy Loss` with a learning rate ($\alpha$) set to `0.005`.

In [8]:
def smol_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1500, input_dim=3630, activation='relu'),
        tf.keras.layers.Dense(47)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.005),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])
    return model
    

In [9]:
BATCH_SIZE = 32
model = smol_model()
model.summary()
history = model.fit(np.array(train_x),np.array(train_y), epochs=3, batch_size=BATCH_SIZE)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:339: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(values, collections.Sequence):


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              5446500   
_________________________________________________________________
dense_1 (Dense)              (None, 47)                70547     
Total params: 5,517,047
Trainable params: 5,517,047
Non-trainable params: 0
_________________________________________________________________


/home/visionteam/python37_env/lib/python3.7/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Train on 112799 samples
Epoch 1/3
112799/112799 [==============================] - 10s 87us/sample - loss: 1.5311 - sparse_categorical_accuracy: 0.6422
Epoch 2/3
112799/112799 [==============================] - 9s 84us/sample - loss: 1.2412 - sparse_categorical_accuracy: 0.7044
Epoch 3/3
112799/112799 [==============================] - 10s 88us/sample - loss: 1.1487 - sparse_categorical_accuracy: 0.7233


# Test the model

In [ ]:
model.evaluate(np.array(test_x), np.array(test_y), batch_size=len(test_x))

# Restart the notebook to free up the `GPU` and `RAM`.

In [ ]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

# Exercise: Log the RAM usage for the two cases.
* How the RAM usage varies when training the model using `.tfrecord` vs direct `NumPy` array.
* How does the speed vary?
* Plot various metrics like `training cost vs epochs`, `training accuracy vs epochs` etc. These metrics can be found in the dictionary `history` returned by the model.